In [2]:
import torch_geometric as geo

In [3]:
import torch

In [4]:
IMDB_B = geo.datasets.TUDataset(root='data', name='IMDB-BINARY')

In [5]:
def idx2adj(data):
  device = data.edge_index.device
  edge_index = torch.zeros(size=(data.num_nodes, data.num_nodes), device=device)
  for i in range(data.edge_index.shape[1]):
      edge_index[data.edge_index[0][i]][data.edge_index[1][i]] = 1
  return edge_index
def adj2idx(edge_index):
  assert edge_index.shape[0] == edge_index.shape[1]
  tmp = []
  for i in range(edge_index.shape[0]):
      for j in range(edge_index.shape[0]):
          if edge_index[i][j] == 1:
              tmp.append([i, j])
  return torch.Tensor(tmp).permute(1,0).to(edge_index.device)

In [6]:
COLLAB = geo.datasets.TUDataset(root='data', name='COLLAB')

Extracting data/COLLAB/COLLAB.zip
Processing...
Done!


In [18]:
numNodes = []
for i in range(IMDB_B.len()):
  numNodes.append(len(list(dict(geo.utils.convert.to_networkx(IMDB_B[i]).degree).values())))

In [20]:
sample = geo.data.Data()
num_nodes = int((sum(numNodes) / len(numNodes)) * 2 * (torch.rand(1)))
adv_adj = torch.ones(size=(num_nodes, num_nodes)).int()
sample.edge_index = adj2idx(adv_adj).long()
sample.num_nodes = num_nodes

if True:
  sample.edge_attr = torch.rand(sample.edge_index.shape[1], )
  sample.edge_attr.requires_grad_()
  bin_attr = (sample.edge_attr + 0.5).int().bool()
  bin_edge0 = sample.edge_index[0].masked_select(bin_attr)
  bin_edge1 = sample.edge_index[1].masked_select(bin_attr)
  sample.x = bin_edge0.bincount().float()
  cl_optim = torch.optim.Adam([sample.edge_attr])
